In [1]:
!pip install nba-api
!pip install requests
!pip install seaborn
!pip install -U scikit-learn

zsh:1: /Users/luishernandez/Desktop/NBStats/myenv/bin/pip: bad interpreter: /home/francisco/Documents/NBA_APP/myenv/bin/python3: no such file or directory
  Using cached nba_api-1.8.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.1-cp313-cp313-macosx_10_13_universal2.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2025.1.31-py3-none-any.whl (166 kB)
Using cached charset_normalizer-3.4.1-cp313-cp313-macosx_10_13_universal2.whl (195 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)
zsh:1: /Users/luishernandez/Desktop/NBStats/myenv/bin/pip: bad interpreter: /home/francisco/Documents/NBA_APP/myenv/bin/python3:

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from typing import List, Dict, Tuple, Union, Optional


In [3]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.endpoints import commonteamroster
from datetime import datetime, timezone
from dateutil import parser
from nba_api.live.nba.endpoints import scoreboard


In [4]:
class Teams():
    def __init__(self, Team_Abrrev, Season):
        self.Team_Abrrev= Team_Abrrev
        self.Season = Season

    def get_team_roster_per_season(teamAbbreviation:str, season:str) -> pd.DataFrame:
        team_details = teams.find_team_by_abbreviation(teamAbbreviation)
        team_id = team_details["id"]
        roster = commonteamroster.CommonTeamRoster(team_id=team_id, season=season)
        roster_data = roster.get_data_frames()[0]
        return roster_data

    def get_team_games_played_per_season(teamAbbreviation:str, season:str) -> pd.DataFrame:
        team_details = teams.find_team_by_abbreviation(teamAbbreviation)
        team_id = team_details["id"]
        game_log = teamgamelog.TeamGameLog(team_id=team_id, season=season)
        game_log = game_log.get_data_frames()[0]
        return game_log
        
    def get_regular_season_teams_matchups_per_season(Team_Abrrev_list:list, season:str) -> pd.DataFrame:
        if type(Team_Abrrev_list) is not list:
            return "Team_abbrev_list must be a list"
        else: 
            teamAbbreviation1 = Team_Abrrev_list[0]
            teamAbbreviation2 = Team_Abrrev_list[1]
            game_log = get_team_games_played_per_season(teamAbbreviation1, season)
            return game_log[(game_log["MATCHUP"] == f"{teamAbbreviation1} @ {teamAbbreviation2}") | 
                            (game_log["MATCHUP"] == f"{teamAbbreviation1} vs. {teamAbbreviation2}")]  
            
    def get_todays_games() -> None:
        f = "{gameId}: {awayTeam} vs. {homeTeam} @ {gameTimeLTZ}" 
        board = scoreboard.ScoreBoard()
        print("ScoreBoardDate: " + board.score_board_date)
        games = board.games.get_dict()
        for game in games:
            gameTimeLTZ = parser.parse(game["gameTimeUTC"]).replace(tzinfo=timezone.utc).astimezone(tz=None)
            print(f.format(gameId=game['gameId'], awayTeam=game['awayTeam']['teamName'], homeTeam=game['homeTeam']['teamName'], gameTimeLTZ=gameTimeLTZ))

In [5]:
Teams.get_team_roster_per_season("LAL",'2024-25')
#Teams.get_team_games_played_per_season("LAL", "2024-25")
#Teams.get_regular_season_teams_matchups_per_season(["OKC", "DAL"], "2022-23")
#Teams.get_todays_games()) <----- Falta cambiar para que en vez de imprimir, retorne...


,TeamID,SEASON,LeagueID,PLAYER,NICKNAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID,HOW_ACQUIRED
0,1610612747,2024,00,Jarred Vanderbilt,Jarred,jarred-vanderbilt,2,F,6-8,214,"APR 03, 1999",25.0,6,Kentucky,1629020,Traded from UTA on 02/09/23
1,1610612747,2024,00,Dalton Knecht,Dalton,dalton-knecht,4,F,6-6,215,"APR 19, 2001",23.0,R,Tennessee,1642261,#17 Pick in 2024 Draft
2,1610612747,2024,00,Cam Reddish,Cam,cam-reddish,5,F-G,6-7,217,"SEP 01, 1999",25.0,5,Duke,1629629,Signed on 07/06/23
3,1610612747,2024,00,Gabe Vincent,Gabe,gabe-vincent,7,G,6-2,200,"JUN 14, 1996",28.0,5,California-Santa Barbara,1629216,Signed on 07/06/23
4,1610612747,2024,00,Bronny James,Bronny,bronny-james,9,G,6-3,210,"OCT 06, 2004",20.0,R,Southern California,1642355,#55 Pick in 2024 Draft
5,1610612747,2024,00,Christian Koloko,Christian,christian-koloko,10,C,7-0,225,"JUN 20, 2000",24.0,1,Arizona,1631132,Signed on 09/16/24
6,1610612747,2024,00,Jaxson Hayes,Jaxson,jaxson-hayes,11,C-F,7-0,220,"MAY 23, 2000",24.0,5,Texas,1629637,Signed on 07/06/23
7,1610612747,2024,00,Maxi Kleber,Maxi,maxi-kleber,14,F,6-10,240,"JAN 29, 1992",33.0,7,Bayern Munich,1628467,Traded from DAL on 02/02/25
8,1610612747,2024,00,Austin Reaves,Austin,austin-reaves,15,G,6-5,197,"MAY 29, 1998",26.0,3,Oklahoma,1630559,Signed on 08/03/21
9,1610612747,2024,00,Dorian Finney-Smith,Dorian,dorian-finney-smith,17,F,6-7,220,"MAY 04, 1993",31.0,8,Florida,1627827,Traded from BKN on 12/29/24


In [6]:

from nba_api.stats.endpoints import teamyearbyyearstats

year_stats = teamyearbyyearstats.TeamYearByYearStats(team_id=1610612747)
year_stats = year_stats.get_data_frames()[0]

In [7]:
year_stats.columns

Index(['TEAM_ID', 'TEAM_CITY', 'TEAM_NAME', 'YEAR', 'GP', 'WINS', 'LOSSES',
       'WIN_PCT', 'CONF_RANK', 'DIV_RANK', 'PO_WINS', 'PO_LOSSES',
       'CONF_COUNT', 'DIV_COUNT', 'NBA_FINALS_APPEARANCE', 'FGM', 'FGA',
       'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB',
       'DREB', 'REB', 'AST', 'PF', 'STL', 'TOV', 'BLK', 'PTS', 'PTS_RANK'],
      dtype='object')

In [8]:
year_stats.tail()

,TEAM_ID,TEAM_CITY,TEAM_NAME,YEAR,GP,WINS,LOSSES,WIN_PCT,CONF_RANK,DIV_RANK,...,OREB,DREB,REB,AST,PF,STL,TOV,BLK,PTS,PTS_RANK
72,1610612747,Los Angeles,Lakers,2020-21,72,42,30,0.583,7,3,...,695,2490,3185,1775,1377,562,1095,386,7887,22
73,1610612747,Los Angeles,Lakers,2021-22,82,33,49,0.402,11,4,...,782,2826,3608,1968,1654,626,1191,425,9192,11
74,1610612747,Los Angeles,Lakers,2022-23,82,43,39,0.524,7,5,...,818,2930,3748,2072,1466,526,1154,378,9608,6
75,1610612747,Los Angeles,Lakers,2023-24,82,47,35,0.573,8,3,...,676,2859,3535,2340,1283,605,1146,453,9679,6
76,1610612747,Los Angeles,Lakers,2024-25,70,43,27,0.614,4,1,...,675,2309,2984,1838,1195,541,979,322,7898,23


In [9]:
year_stats[year_stats["YEAR"] == "2010-11"]

,TEAM_ID,TEAM_CITY,TEAM_NAME,YEAR,GP,WINS,LOSSES,WIN_PCT,CONF_RANK,DIV_RANK,...,OREB,DREB,REB,AST,PF,STL,TOV,BLK,PTS,PTS_RANK
62,1610612747,Los Angeles,Lakers,2010-11,82,57,25,0.695,2,1,...,989,2616,3605,1801,1555,602,1073,422,8321,9


In [10]:
info = year_stats[62:]

In [15]:
info.columns

Index(['TEAM_ID', 'TEAM_CITY', 'TEAM_NAME', 'YEAR', 'GP', 'WINS', 'LOSSES',
       'WIN_PCT', 'CONF_RANK', 'DIV_RANK', 'PO_WINS', 'PO_LOSSES',
       'CONF_COUNT', 'DIV_COUNT', 'NBA_FINALS_APPEARANCE', 'FGM', 'FGA',
       'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB',
       'DREB', 'REB', 'AST', 'PF', 'STL', 'TOV', 'BLK', 'PTS', 'PTS_RANK'],
      dtype='object')

In [21]:
info = info[["TEAM_ID", "TEAM_NAME", "YEAR", "GP", "WINS", "LOSSES"]]
info

,TEAM_ID,TEAM_NAME,YEAR,GP,WINS,LOSSES
62,1610612747,Lakers,2010-11,82,57,25
63,1610612747,Lakers,2011-12,66,41,25
64,1610612747,Lakers,2012-13,82,45,37
65,1610612747,Lakers,2013-14,82,27,55
66,1610612747,Lakers,2014-15,82,21,61
67,1610612747,Lakers,2015-16,82,17,65
68,1610612747,Lakers,2016-17,82,26,56
69,1610612747,Lakers,2017-18,82,35,47
70,1610612747,Lakers,2018-19,82,37,45
71,1610612747,Lakers,2019-20,71,52,19


In [22]:
from json import loads, dumps
info.to_json('lakers_stats.json', orient = 'split', compression = 'infer', index = 'true')

In [23]:
read = pd.read_json("lakers_stats.json",orient ='split', compression = 'infer')

In [24]:
read.reset_index(inplace=True)

In [25]:
read.drop(columns="index")

,TEAM_ID,TEAM_NAME,YEAR,GP,WINS,LOSSES
0,1610612747,Lakers,2010-11,82,57,25
1,1610612747,Lakers,2011-12,66,41,25
2,1610612747,Lakers,2012-13,82,45,37
3,1610612747,Lakers,2013-14,82,27,55
4,1610612747,Lakers,2014-15,82,21,61
5,1610612747,Lakers,2015-16,82,17,65
6,1610612747,Lakers,2016-17,82,26,56
7,1610612747,Lakers,2017-18,82,35,47
8,1610612747,Lakers,2018-19,82,37,45
9,1610612747,Lakers,2019-20,71,52,19
